<a href="https://colab.research.google.com/github/hyunseo-hy/Python-for-Finance-Practice/blob/main/WTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
#Spot=현물, Front=최근월물, Second=차근월물
df1 = pd.read_excel("/content/11.2 WTI.xlsx","Spot", index_col="Date")
df2 = pd.read_excel("/content/11.2 WTI.xlsx","Front", index_col="Date")
df3 = pd.read_excel("/content/11.2 WTI.xlsx","Second", index_col="Date")

df = pd.concat([df1,df2,df3],axis=1)
print(df.head())

df_e = pd.read_excel("/content/11.2 WTI.xlsx","Expiry")

             Spot  Front  Second
Date                            
2017-12-28  59.84  59.84   59.87
2017-12-27  59.64  59.64   59.69
2017-12-26  59.97  59.97   60.00
2017-12-22  58.42  58.47   58.54
2017-12-21  58.33  58.36   58.40


In [5]:
df=df.fillna(method="ffill")
df=df.fillna(method="bfill")

/tmp/ipython-input-2706507583.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df=df.fillna(method="ffill")
/tmp/ipython-input-2706507583.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df=df.fillna(method="bfill")


In [6]:
df_e[120:125]
expiry_date = pd.Series(pd.to_datetime(df_e["Last Trade"]))
expiry_date

/tmp/ipython-input-3683118747.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  expiry_date = pd.Series(pd.to_datetime(df_e["Last Trade"]))


,Last Trade
0,2000-01-20
1,2000-02-22
2,2000-03-21
3,2000-04-19
4,2000-05-22
...,...
288,2024-11-20
289,2025-05-20
290,2025-11-20
291,2026-05-19


In [7]:
df["Expiry"] = np.nan
print(df.head())
print(df_e)

             Spot  Front  Second  Expiry
Date                                    
2017-12-28  59.84  59.84   59.87     NaN
2017-12-27  59.64  59.64   59.69     NaN
2017-12-26  59.97  59.97   60.00     NaN
2017-12-22  58.42  58.47   58.54     NaN
2017-12-21  58.33  58.36   58.40     NaN
    Ticker      Month Last Trade  Last Price
0    CLG00 2000-02-01   01/20/00       29.66
1    CLH00 2000-03-01   02/22/00       29.62
2    CLJ00 2000-04-01   03/21/00       28.00
3    CLK00 2000-05-01   04/19/00       27.35
4    CLM00 2000-06-01   05/22/00       28.61
..     ...        ...        ...         ...
288   CLZ4 2017-12-24   11/20/24         NaN
289   CLM5 2017-06-25   05/20/25         NaN
290   CLZ5 2017-12-25   11/20/25         NaN
291   CLM6 2017-06-26   05/19/26         NaN
292   CLZ6 2017-12-26   11/20/26         NaN

[293 rows x 4 columns]


In [8]:
for d in df.index:
  if d in pd.to_datetime(expiry_date.values):
    df.loc[d, "Expiry"] = 1
print(df[5:10])

             Spot  Front  Second  Expiry
Date                                    
2017-12-20  58.02  58.09   58.13     NaN
2017-12-19  57.46  57.46   57.56     1.0
2017-12-18  57.16  57.16   57.22     NaN
2017-12-15  57.30  57.30   57.33     NaN
2017-12-14  57.04  57.04   57.08     NaN


In [9]:
#without looping
new_expire = pd.to_datetime(expiry_date.values)
df["Expiry"] = df.index.isin(new_expire).astype(int)
print(df)
print(sum(df["Expiry"]==1))

             Spot  Front  Second  Expiry
Date                                    
2017-12-28  59.84  59.84   59.87       0
2017-12-27  59.64  59.64   59.69       0
2017-12-26  59.97  59.97   60.00       0
2017-12-22  58.42  58.47   58.54       0
2017-12-21  58.33  58.36   58.40       0
...           ...    ...     ...     ...
2010-01-08  82.75  82.75   83.30       0
2010-01-07  82.66  82.66   83.19       0
2010-01-06  83.18  83.18   83.75       0
2010-01-05  81.77  81.77   82.41       0
2010-01-04  81.51  81.51   82.12       0

[2014 rows x 4 columns]
96


In [27]:
price_now = float(df.loc[pd.to_datetime("2017-12-28")]["Spot"])
price_before = float(df.loc[pd.to_datetime("2010-01-04")]["Spot"])
profit = (price_now-price_before)*1000
print(f"손익(현재가-초기가): {"{:,.2f}".format(profit)}")

손익(현재가-초기가): -21,670.00


In [19]:
roll_over_cost_acc = 0
for d in df.index:
  if (df.loc[d].Expiry==1):
    roll_over_cost = df.loc[d].Second-df.loc[d].Front
    roll_over_cost_acc += roll_over_cost
print(f"누적 롤오버 비용: {"{:,.2f}".format(roll_over_cost_acc)}")

누적 롤오버 비용: 40.68


In [26]:
price_now = df.iloc[0].Front
price_before = df.iloc[len(df)-1].Front
profit = (price_now-price_before-roll_over_cost_acc)*1000
print(f"누적 손익: {"{:,.2f}".format(profit)}")

누적 손익: -62,350.00
